In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import os

# Parameters
model_type = "mobilenet_v2_small"
image_size = [256, 256]
valid_size = 0.3
batch_size = 128
learning_rate = 0.0003
learning_rate_decay_factor = 0.95
num_of_output_nodes = 2

# Paths
train_dir = "train"  

# Data Preparation
def prepare_data():
    # Image Data Generator with Augmentation
    datagen = ImageDataGenerator(
        rescale=1.0 / 255,  
        validation_split=valid_size,  
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    )

    # Training Data
    train_data = datagen.flow_from_directory(
        train_dir,
        target_size=image_size,
        batch_size=batch_size,
        class_mode='binary',
        subset='training'
    )

    # Validation Data
    valid_data = datagen.flow_from_directory(
        train_dir,
        target_size=image_size,
        batch_size=batch_size,
        class_mode='binary',
        subset='validation'
    )

    return train_data, valid_data

# Model Building
def build_model():
    # Load MobileNetV2 (small)
    base_model = MobileNetV2(
        input_shape=image_size + [3],  # Image size + 3 channels (RGB)
        include_top=False,  
        weights='imagenet'  
    )

    # Freeze the base model
    base_model.trainable = False

    # Add custom layers for classification
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.3)(x) 
    x = Dense(128, activation='relu')(x)  
    x = Dropout(0.3)(x) 
    outputs = Dense(num_of_output_nodes, activation='softmax')(x)  

    # Create the model
    model = Model(inputs=base_model.input, outputs=outputs)

    return model

# Compile Model
def compile_model(model):

    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=learning_rate,
        decay_steps=1000,
        decay_rate=learning_rate_decay_factor
    )

    model.compile(
        optimizer=Adam(learning_rate=lr_schedule),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# Train Model
def train_model(model, train_data, valid_data):

    # Early stopping and checkpoint callbacks
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    )

    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath="best_model.h5",
        monitor='val_loss',
        save_best_only=True
    )

    # Train the model
    history = model.fit(
        train_data,
        validation_data=valid_data,
        epochs=30,
        callbacks=[early_stopping, checkpoint],
        verbose=1
    )

    return history

# Main Function
def main():

    train_data, valid_data = prepare_data()

    # Build and compile the model
    model = build_model()
    model = compile_model(model)

    # Train the model
    history = train_model(model, train_data, valid_data)

    # Save the final model
    model.save("hair_length_classifier.h5")
    print("Model training complete and saved as 'hair_length_classifier.h5'")

    # Evaluate the model on validation data
    val_loss, val_acc = model.evaluate(valid_data, verbose=1)
    print(f"Validation Accuracy: {val_acc * 100:.2f}%")

if __name__ == "__main__":
    main()


Found 14610 images belonging to 2 classes.
Found 6260 images belonging to 2 classes.


C:\Users\Piya\AppData\Local\Temp\ipykernel_3884\1761708524.py:64: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(
C:\Users\Piya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
457/457 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9305 - loss: 0.1812

KeyboardInterrupt: 

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

def preprocess_new_image(img_path):
    img = image.load_img(img_path, target_size=image_size)
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0  # Rescale
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Load saved model
model = tf.keras.models.load_model('hair_length_classifier.h5')
img_array = preprocess_new_image(r'C:\Users\Piya\Desktop\My Codes\python code\hair_length_pred\image to test\Chris_hemsjpg.jpg')
prediction = model.predict(img_array)
class_label = int(prediction[0] > 0.5)
class_names = ['short_hair', 'long_hair']
print(f"Predicted hair length: {class_names[class_label]}")

In [2]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.Collecting tensorflow
  Using cached tensorflow-2.18.0-cp310-cp310-win_amd64.whl (7.5 kB)
  Using cached tensorflow_intel-2.18.0-cp310-cp310-win_amd64.whl (390.0 MB)
     ---------------------------------------- 15.9/15.9 MB 1.1 MB/s eta 0:00:00
  Using cached keras-3.8.0-py3-none-any.whl (1.3 MB)
  Using cached tensorflow_io_gcs_filesystem-0.31.0-cp310-cp310-win_amd64.whl (1.5 MB)
  Using cached termcolor-2.5.0-py3-none-any.whl (7.8 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl (26.4 MB)
  Using cached gast-0.6.0-py3-none-any.whl (21 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached grpcio-1.69.0-cp310-cp310-win_amd64.whl (4.4 MB)
  Using cached tensorboard-2.18.0-py3-none-any.whl (5.5 MB)
  Using cached h5py-3.12.1-cp310-cp310-win_amd64.whl (3.0 MB)
  Using cached ml_dtypes-0.4.1-cp310-cp310-win_amd64.whl (126 kB)
  